In [1]:
import pandas as pd

In [2]:
takeup = 'data/asec19_takeup.sas7bdat'
early = 'data/hhpub18early_18par.sas7bdat'

In [3]:
take_up = pd.read_sas(takeup)
early_ = pd.read_sas(early)

/home/bobbyhuck/anaconda3/lib/python3.7/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
/home/bobbyhuck/anaconda3/lib/python3.7/site-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


In [6]:
take_up.shape

(180101, 34)

In [7]:
early_.shape

(5000, 127)

## **1. Sign-in Sign-out Logs**

Application logs are used in analysis of interactions with an application and may be used to detect specific actions.  

A log file is provided as a string aray where each entry is in the form "*user_id timestamp action*". Each of the values is separated by a space.  

* Both *user_id* and *timestamp* consist only of digits, are at most 9 digits long and start with a non-zero digit.  
* *timestamp* represents the time in seconds since the application was last launched.  
* *action* will be either "sign-in" or "sign-out".  

Give a log with entries in no particular order, return an array of strings that denote *user_id*'s of users who signed out in *maxSpan* seconds or less after signing in.  

**Example**  
n = 7  
logs = ['30 99 sign-in', '30 105 sign-out', '12 100 sign-in', '20 80 sign-in', '12 120 sign-out', '20 101 sign-out', '21 110 sign-in']  
maxSpan = 20  

ID Sign in Sign out Time delta
__ _______ ________ __________
30 99      105      6  
12 100     120      20  
20 80      101      21  
21 110  

The users with id's 30 and 12 were not signed in for more than *maxSpan* = 20 seconds. In sorted numerical order, the return array is ['12', '30']  

**Function Description**  

Complete the function *processLogs* in the editor below.  

The function has the following parameters(s):  
string *logs[n]*: each *logs[i]* denotes teh ith entry in the logs  
int *maxSpan*: the maximum difference in seconds between when a user logs in and logs out for the user to be included in the result    
Returns:  
*string*: a string array of user id's, sorted ascending by numeric value

In [ ]:
import numpy as np
import pandas as pd

def processLogs(logs, maxSpan):

    data_array = np.array(logs)

    df = pd.DataFrame(data, columns=['ID', 'Sign in', 'Sign out', 'Time Delta'])

Could make a dict to store id's and if 'sign-in' then note initial value. if 'sign-out' then create another dict where key, val are in-out figures

Keep in mind edge cases in which 'sign-out' of a particular id could come before 'sign-in'

In [8]:
import numpy as np
import pandas as pd

def processLogs(logs, maxSpan):

    data_array = np.array(logs)

    #df = pd.DataFrame(data_array, columns=['ID', 'Sign in', 'Sign out', 'Time Delta'])
    
    return data_array

In [9]:
logs = ['30 99 sign-in', '30 105 sign-out', '12 100 sign-in', '20 80 sign-in', '12 120 sign-out', '20 101 sign-out', '21 110 sign-in']

maxSpan = 20

processLogs(logs, maxSpan)

array(['30 99 sign-in', '30 105 sign-out', '12 100 sign-in',
       '20 80 sign-in', '12 120 sign-out', '20 101 sign-out',
       '21 110 sign-in'], dtype='<U15')

## **2. Optimal Meeting Slots**  

**Problem Description**  
You need to implememnt a meeting scheduler that uses the schedules of the employees to find a specific number of optimal meeting slots where the largest number of employees can attend.  

Input:  
1. An integer *num_slots* representing the total number of meeting slots to return  
2. A list of lists of strings representing each employee's list of meetings (parameter employee_schedules). Meetings are specified as strings in th format "XX:XX-YY:YY" with XX:XX as the beginning time of the meeting and YY:YY as the end time of the meeting, all in zero-padded 24-hour time. The very end of the day can be expressed as 24:00 for simplicity

Example Input
3  
[
["00:00-05:30"],
["00:00-05:00", "08:00-22:00"],
["00:00-12:30"]
]

Output:  
A list of strings representing the optima meeting times in earlies-to-latest order. An unsorted output will count as incorrect. Optimal is defined as follows:
1. Meetings with the highest number of employees should be prioritized.
2. For meetings with the same number of employees that can attend, the meeting with the earlier start time should be prioritized.

Note:  
If the number of possible valid meeting slots that can be constructed is less than the requested amount, *num_slots*, the function should return and empty list of meeting slots([])

Constraints and assumptions:  

* Input employee meeting schedules will always consist of valid meetings and in the specified format
* Valid outputted meetings do not overlap
* Two meetings: 12:30-13:00 and 13:00-14:00 do not overlap, where the first is a meeting that ends at 1:00pm and the second is a meeting that starts at 1:00pm
* Any employee who is not in a meeting at any given time is considered "free" and can be scheduled for a meeeting. Inversely, an employee who is scheduled for a meeting at a given time cannot attend any other mettings at that time. This is also true for meeting slots.
* Meetings are scheduled at 15 min intervals (i.e. 07:15 is a valid start time or end time for a meeting, 18:47 is not). As a consequence, an meetings scheduled must be at least 15 min in length.  
* A meeting cannot be scheduled for a time if fewer than 2 employees are available at that time.  
* Importantly, meeting slots must be selected for the maximum amount of time possible wihtout decreasing the number of employees that can attend while still generating valid, optimal meeting slots. In other words, if some set of employees are all available from 10:00-10:30, the valid meeting slot would be 10:00-10:30, 2 meeting slots from 10:00-10:15 and 10:15-10:30 would not be valid and distinct.
* If 3 other coworkers (for a total of 6 participants) are available during a chunk of the potential meeting slot (i.e. 10:15-10:30), to still output the most optimal shiwl still valid slots, two valid slots (due to valid slots not overlapping) can be formed. One with 6 participants 10:15-10:30 (more optimal) and one with 3 participants from 10:00-10:15 (less optimal).

Example Output:
["05:30-08:00", "12:30-22:00", "22:00-24:00"]  

The value of *num_slots* is 3 and the 3 valid optimal meeting slots were returned. The 3rd meeting slot has 3 available participants, and the first two each only had two available participants. The output is sorted in earliest-to-latest order. The full available, distinct meeting slots are returned not truncated to allow for more meetings (such as incorrectly splitting up "22:00-24:00" into shorted contiguous meeting slots).

Hint:  
You might find it helpful to think aobu the problem by considering each 15 min time slot as a discrete unit, where a given employee may ore may not have availability.

In [ ]:
def fing_meeting_slots(num_slots, employee_schedules):
    
    pass